# Using GRU Model for testing Ambig QA

In [2]:
import numpy as np
import pandas as pd 
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [4]:
!ls

Break-dataset.zip  sample_data


In [5]:
!unzip /content/Break-dataset.zip

Archive:  /content/Break-dataset.zip
   creating: Break-dataset/logical-forms/
  inflating: Break-dataset/logical-forms/dev.csv  
  inflating: Break-dataset/logical-forms/test.csv  
  inflating: Break-dataset/logical-forms/train.csv  
   creating: Break-dataset/QDMR/
  inflating: Break-dataset/QDMR/dev.csv  
  inflating: Break-dataset/QDMR/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/test.csv  
  inflating: Break-dataset/QDMR/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/train.csv  
  inflating: Break-dataset/QDMR/train_lexicon_tokens.json  
   creating: Break-dataset/QDMR-high-level/
  inflating: Break-dataset/QDMR-high-level/dev.csv  
  inflating: Break-dataset/QDMR-high-level/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/test.csv  
  inflating: Break-dataset/QDMR-high-level/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/train.csv  
  inflating: Break-dataset/QDMR-high-level/train_lexicon_tokens.json  


In [9]:
df = pd.read_csv('/content/Break-dataset/QDMR/train.csv')


In [10]:
df.head(5)

,question_id,question_text,decomposition,operators,split
0,ACADEMIC_train_0,return me the homepage of PVLDB .,return homepages ;return #1 of PVLDB,"['select', 'filter']",train
1,ACADEMIC_train_1,"return me the homepage of "" H. V. Jagadish "" .",return homepages ;return #1 of H. V. Jagadish,"['select', 'filter']",train
2,ACADEMIC_train_10,"return me the number of references of "" Making...",return references ;return #1 of Making databa...,"['select', 'filter', 'aggregate']",train
3,ACADEMIC_train_100,return me the number of papers in VLDB confere...,return papers ;return #1 in VLDB conference ;r...,"['select', 'filter', 'filter', 'aggregate']",train
4,ACADEMIC_train_101,return me the number of authors who have paper...,return papers ;return #1 containing keyword Re...,"['select', 'filter', 'project', 'aggregate']",train


In [11]:
df.columns

Index(['question_id', 'question_text', 'decomposition', 'operators', 'split'], dtype='object')

In [ ]:
df.columns

Index(['viewed_doc_titles', 'used_queries', 'annotations', 'nq_answer', 'id',
       'nq_doc_title', 'question'],
      dtype='object')

In [21]:
data = df.rename(columns={'question_text':'Question','decomposition':'Answer'})

data.to_csv('train_data.csv',index=False)
#column_names = ['id','question','nq_answer','nq_doc_title']
#data = df.reindex(columns= column_names)

In [27]:
df1 = data[['Question','Answer']]

In [28]:
#df1 = pd.read_csv('train_data.csv')
df1.head(5)

,Question,Answer
0,return me the homepage of PVLDB .,return homepages ;return #1 of PVLDB
1,"return me the homepage of "" H. V. Jagadish "" .",return homepages ;return #1 of H. V. Jagadish
2,"return me the number of references of "" Making...",return references ;return #1 of Making databa...
3,return me the number of papers in VLDB confere...,return papers ;return #1 in VLDB conference ;r...
4,return me the number of authors who have paper...,return papers ;return #1 containing keyword Re...


In [ ]:
data
#data.dropna(how='any')

,question,nq_answer
0,When did the simpsons first air on television?,"[December 17 , 1989]"
1,Who played george washington in the john adams...,[David Morse]
2,What is the legal age of marriage in usa?,"[18, Nebraska ( 19 ), Mississippi ( 21 )]"
3,Who starred in barefoot in the park on broadway?,"[Robert Redford, Elizabeth Ashley]"
4,When did the manhattan project began and end?,[From 1942 to 1946]
...,...,...
10031,When do the summer holidays start for schools?,[between late May and mid-June]
10032,Who is the band in the movie 10 things i hate ...,[Save Ferris]
10033,Who was the last person in the uk to be executed?,[1964]
10034,Who does wonder woman end up with in the comics?,[Steve Trevor]


In [29]:
TEXT = torchtext.data.Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('question', TEXT), ('nq_answer', TEXT)]

In [30]:
# Create dataset
start = time.time()
dataset = torchtext.data.TabularDataset(
    path='train_data.csv',
    format='CSV',
    fields=fields,
    skip_header=True
)
end = time.time()
print(f'Duration: {end - start}')

Duration: 10.129972219467163


In [31]:
# Train/val split
train_data, valid_data = dataset.split(split_ratio=[0.85, 0.15])
print(len(train_data), len(valid_data))

37673 6648


In [32]:
vars(train_data.examples[0])

{'nq_answer': ['how',
  'many',
  'times',
  'did',
  'the',
  'peasants',
  'and',
  'city',
  'dwellers',
  'defeat',
  'the',
  'troops',
  'of',
  'the',
  'french',
  'count',
  'of',
  'armagnac',
  '?'],
 'question': ['drop_train_history_274_d2589bf8',
  '-',
  '62c6',
  '-',
  '4e41',
  '-',
  '867d-524123acd218']}

In [33]:
MAX_VOCAB_SIZE = 10_000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)

print(f'Size of vocab: {len(TEXT.vocab)}')


Size of vocab: 10004


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [35]:
BATCH_SIZE = 128

train_iterator, valid_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data),
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.question),
    device = device
)

In [36]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden


In [37]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [38]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [39]:


INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)



In [40]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(128, 256)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(384, 256)
    (fc_out): Linear(in_features=640, out_features=10004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [41]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,763,092 trainable parameters


In [42]:
optimizer = optim.Adam(model.parameters())

In [43]:


TRG_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)



In [44]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question
        trg = batch.nq_answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [45]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            trg = batch.nq_answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [46]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [47]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 0s
	Train Loss: 5.311 | Train PPL: 202.478
	 Val. Loss: 5.473 |  Val. PPL: 238.194
Epoch: 02 | Time: 1m 0s
	Train Loss: 4.464 | Train PPL:  86.806
	 Val. Loss: 5.397 |  Val. PPL: 220.634
Epoch: 03 | Time: 1m 0s
	Train Loss: 4.260 | Train PPL:  70.793
	 Val. Loss: 5.396 |  Val. PPL: 220.464
Epoch: 04 | Time: 1m 1s
	Train Loss: 4.082 | Train PPL:  59.285
	 Val. Loss: 5.488 |  Val. PPL: 241.883
Epoch: 05 | Time: 1m 1s
	Train Loss: 3.987 | Train PPL:  53.918
	 Val. Loss: 5.485 |  Val. PPL: 241.001
Epoch: 06 | Time: 1m 1s
	Train Loss: 3.915 | Train PPL:  50.133
	 Val. Loss: 5.516 |  Val. PPL: 248.736
Epoch: 07 | Time: 1m 0s
	Train Loss: 3.858 | Train PPL:  47.351
	 Val. Loss: 5.468 |  Val. PPL: 237.011
Epoch: 08 | Time: 1m 0s
	Train Loss: 3.778 | Train PPL:  43.748
	 Val. Loss: 5.507 |  Val. PPL: 246.471
Epoch: 09 | Time: 1m 0s
	Train Loss: 3.751 | Train PPL:  42.568
	 Val. Loss: 5.510 |  Val. PPL: 247.164
Epoch: 10 | Time: 1m 0s
	Train Loss: 3.712 | Train PPL:  40.925
